# phData - Snowflake Cortex HOL

## 1. Overview

To reduce hallucinations (i.e. incorrect responses), LLMs can be combined with private datasets. Today, the most common approach for reducing hallucinations without having to change the model (e.g. fine-tuning) is the Retrieval Augmented Generation (RAG) framework. RAG allows you to "ground" the model's responses by making a set of relevant documents available to the LLM as context in the response.

In this quickstart we will show you how to quickly and securely build a full-stack RAG application in Snowflake without having to build integrations, manage any infrastructure or deal with security concerns with data moving outside of the Snowflake governance framework.

We will show you how easy it is to implement RAG via a chat assistant that knows everything about smart devices. This assistant can be really useful for your not so tech-savvy friend or relative that is always asking you questions about their electronics. To make the assistant an expert in a smart devices, we are going to give it access to a few User Manuals. This template can easily be adapted to other documents that may be more interesting to you whether its financial reports, research documents or anything else!

Along the way, we will also share tips on how you could turn what may seem like a prototype into a production pipeline by showing you how to automatically process new documents as they are uploaded as well as learn about relevant Snowflake functionality to consider for additional enhancements.

### What You Will Build

The final product includes an application that lets users test how the LLM responds with and without the context document(s) to show how RAG can address hallucinations.

### What You Will Learn

How to create functions that use Python libraries using Snowpark

How to generate embeddings, run semantic search and use LLMs using serverless functions in Snowflake Cortex

How to build a front-end with Python using Streamlit in Snowflake

Optional: How to automate data processing pipelines using directory tables, Streams and Task

### Prerequisites

Snowflake account in a cloud region where Snowflake Cortex LLM functions are supported

Check LLM availability to help you decide where you want to create your snowflake account

A Snowflake account with Anaconda Packages enabled by ORGADMIN.

Snowflake Cortex vector functions for semantic distance calculations along with VECTOR as a data type enabled.

## 2. Organize Documents and Create Pre-Processing Functions

In Snowflake, databases and schemas are used to organize and govern access to data and logic. Let´s start by getting a few documents locally and then create a database that will hold the PDFs, the functions that will process (extract and chunk) those PDFs and the table that will hold the text embeddings.

### Step 1. Download example documents

Let's download a few documents we have created about bikes. In those documents we have added some very specific information about those ficticious models. You can always add more or use a different type of documents that you want to try asking questions against.

### Step 2. Open a new Worksheet

Relevant documentation: Creating Snowflake Worksheets.

### Step 3. Create a database and a schema

Run the following code inside your newly created worksheet:

In [ ]:
-- CREATE DATABASE CC_QUICKSTART_CORTEX_DOCS;
-- CREATE SCHEMA DATA;
USE CC_QUICKSTART_CORTEX_DOCS.DATA;

### Step 4. Create a table function that will read the PDF documents and split them in chunks

We will be using the PyPDF2 and Langchain Python libraries to accomplish the necessary document processing tasks. Because as part of Snowpark Python these are available inside the integrated Anaconda repository, there are no manual installs or Python environment and dependency management required.

Create the function by running the following query inside your worksheet:

In [ ]:
create or replace function pdf_text_chunker(file_url string)
returns table (chunk varchar)
language python
runtime_version = '3.9'
handler = 'pdf_text_chunker'
packages = ('snowflake-snowpark-python','PyPDF2', 'langchain')
as
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from snowflake.snowpark.files import SnowflakeFile
import PyPDF2, io
import logging
import pandas as pd

class pdf_text_chunker:

    def read_pdf(self, file_url: str) -> str:
    
        logger = logging.getLogger("udf_logger")
        logger.info(f"Opening file {file_url}")
    
        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())
            
        reader = PyPDF2.PdfReader(buffer)   
        text = ""
        for page in reader.pages:
            try:
                text += page.extract_text().replace('\n', ' ').replace('\0', ' ')
            except:
                text = "Unable to Extract"
                logger.warn(f"Unable to extract from file {file_url}, page {page}")
        
        return text

    def process(self,file_url: str):

        text = self.read_pdf(file_url)
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 4000, #Adjust this as you see fit
            chunk_overlap  = 400, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )
    
        chunks = text_splitter.split_text(text)
        df = pd.DataFrame(chunks, columns=['chunks'])
        
        yield from df.itertuples(index=False, name=None)
$$;

### Step 5. Create a Stage with Directory Table where you will be uploading your documents:

In [ ]:
create or replace stage docs ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

### Step 6. Upload documents to your staging area

- Select Data on the left of Snowsight

- Click on your database CC_QUICKSTART_CORTEX_DOCS

- Click on your schema DATA

- Click on Stages and select DOCS

- On the top right click on the +Files botton

- Drag and drop the two PDF files you downloaded

### Step 7. Check files has been successfully uploaded

Run this query to check what documents are in the staging area:


In [ ]:
ls @docs;